# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Apache Kafka Streaming (This particular notebook is part of Assignment 7)

### Install Confluent kafka

In [ ]:
!pip install confluent_kafka

Specify your own `BROKERS`, `USERNAME`, `PASSWORD` and `TOPIC` in the below script files.

### Example 1: Send and receive messages

Here we create two files one is `producer1.py` and another one is `consumer1.py`(in Consumer notebook). Producer will send messages to a topic and consumer will read these messages in real-time from that particular topic and displays the message along with its word count and an `ALERT` message if the number of words exceeds 6.

#### Write Consumer file

Here we create a consumer file that will read messages from the specified `topic`.

In [ ]:
%%writefile consumer1.py

import sys
import os

from confluent_kafka import Consumer, KafkaException, KafkaError

# Specify BROKERS, USERNAME, PASSWORD and TOPIC 
# Use the same credentials from Assignment 7
brokers = "" 
username = ""
password = ""
topic = ""

# Set the path for the user-defined modules so that they can be directly imported into the python program
os.environ['CLOUDKARAFKA_BROKERS']= brokers
os.environ['CLOUDKARAFKA_USERNAME']= username
os.environ['CLOUDKARAFKA_PASSWORD']= password
os.environ['CLOUDKARAFKA_TOPIC']= topic

if __name__ == '__main__':
    topics = os.environ['CLOUDKARAFKA_TOPIC'].split(",")

    # Consumer configuration
    conf = {
        'bootstrap.servers': os.environ['CLOUDKARAFKA_BROKERS'],                # Specify kafka servers
        'group.id': "%s-consumer" % os.environ['CLOUDKARAFKA_USERNAME'],
        'session.timeout.ms': 6000,                                             # The consumer sends periodic heartbeats to indicate its liveness to the broker
        'default.topic.config': {'auto.offset.reset': 'smallest'},
        'security.protocol': 'SASL_SSL',
	      'sasl.mechanisms': 'SCRAM-SHA-256',
        'sasl.username': os.environ['CLOUDKARAFKA_USERNAME'],
        'sasl.password': os.environ['CLOUDKARAFKA_PASSWORD']
    }

    c = Consumer(**conf)
    c.subscribe(topics)
    try:
        while True:                       # read data continuously
            msg = c.poll(timeout=1.0)
            if msg is None:
                continue
            if msg.error():
                # Error or event
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    # Error
                    raise KafkaException(msg.error())
            else:
                # Proper message
                # sys.stderr.write('%% %s [%d] at offset %d with key %s:\n' % (msg.topic(), msg.partition(), msg.offset(), str(msg.key())))
                msg_ = msg.value()
                n = len(msg_.split())
                if n<=6:
                  print(msg.value())
                  print('Number of words: %d'%n)
                if n>6:
                  print('ALERT: Word limit exceeds!!')
                print("")

    except KeyboardInterrupt:
        sys.stderr.write('%% Aborted by user\n')

    # Close down consumer to commit final offsets.
    c.close()

#### Run consumer file

The consumer will keep on running and will show output whenever the producer sends some data to that topic.

<font color='blue'>Before executing the below cell ensure that you created the CloudKarafka account and specified the credentials.</font>

In [ ]:
!python consumer1.py

For next example **create a new topic** on CloudKarafka and use its topic name. To create a topic, please refer to step 11 in this [document](https://cdn.iisc.talentsprint.com/CDS/CloudKarafka.pdf).

### Example 2: Compute the rolling mean of last three insertions

Here we create two files one is `producer2.py` and another one is `consumer2.py`(in Consumer notebook). Producer will send data to a topic and consumer will read these records in real-time from that particular topic and displays the rolling mean of the last three insertions. Only the added numbers will be displayed for the first two insertions.

#### Write Consumer file

Here we create a consumer file that will read messages from the specified `topic`.

In [ ]:
%%writefile consumer2.py

import sys
import os

from confluent_kafka import Consumer, KafkaException, KafkaError

# Specify BROKERS, USERNAME, PASSWORD and TOPIC
# Use the same credentials from Assignment 7
brokers = "" 
username = ""
password = ""
topic = ""

# Set the path for the user-defined modules so that they can be directly imported into the python program
os.environ['CLOUDKARAFKA_BROKERS']= brokers
os.environ['CLOUDKARAFKA_USERNAME']= username
os.environ['CLOUDKARAFKA_PASSWORD']= password
os.environ['CLOUDKARAFKA_TOPIC']= topic

if __name__ == '__main__':
    topics = os.environ['CLOUDKARAFKA_TOPIC'].split(",")

    # Consumer configuration
    conf = {
        'bootstrap.servers': os.environ['CLOUDKARAFKA_BROKERS'],                # Specify kafka servers
        'group.id': "%s-consumer" % os.environ['CLOUDKARAFKA_USERNAME'],
        'session.timeout.ms': 6000,                                             # The consumer sends periodic heartbeats to indicate its liveness to the broker
        'default.topic.config': {'auto.offset.reset': 'smallest'},
        'security.protocol': 'SASL_SSL',
	      'sasl.mechanisms': 'SCRAM-SHA-256',
        'sasl.username': os.environ['CLOUDKARAFKA_USERNAME'],
        'sasl.password': os.environ['CLOUDKARAFKA_PASSWORD']
    }

    c = Consumer(**conf)
    c.subscribe(topics)
    numbers = []
    count = 1
    window = 3                              # rolling window
    try:
        while True:                    # read data continuously
            msg = c.poll(timeout=1.0)
            if msg is None:
                continue
            if msg.error():
                # Error or event
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    # Error
                    raise KafkaException(msg.error())
            else:
                # Proper message
                # sys.stderr.write('%% %s [%d] at offset %d with key %s:\n' % (msg.topic(), msg.partition(), msg.offset(), str(msg.key())))
                msg_ = msg.value()                             # read data 
                num = float(msg_.decode("utf-8"))              # decode from bytes and convert it into float type
                numbers.append(num)                            # add current data to list
                if count<window:
                  print(num)
                if count>=window:
                  print(num)
                  mean = sum(numbers[-3:])/3                   # calculate rolling mean
                  print("mean of last three insertions: {}".format(mean))
                print("")
                count += 1

    except KeyboardInterrupt:
        sys.stderr.write('%% Aborted by user\n')

    # Close down consumer to commit final offsets.
    c.close()

#### Run Consumer file

The consumer will keep on running and will show output whenever the producer sends some data to that topic.

<font color='blue'>Before executing the below cell ensure that you created the CloudKarafka account and specified the credentials.</font>

In [ ]:
!python consumer2.py